In [3]:
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [1]:


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.1

/home/ashmat98/.miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Sample

In [3]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


## TensorBoard

In [4]:
writer = tf.summary.FileWriter('log/')
writer.add_graph(tf.get_default_graph())

### run this command to see computational graph
`$ tensorboard --logdir log/`

You should see something like this

![title](img/graph.1.png)

## Session

In [ ]:
sess = tf.Session(config=config)

In [ ]:
# evaluate total
print(sess.run(total))

In [ ]:
# You can pass multiple tensors to tf.Session.run
print(sess.run({'ab':(a, b), 'total':total}))

the following code calls `tf.random_uniform` to produce a `tf.Tensor` that generates a random 3-element vector (with values in \[0,1))

The result shows a different random value on each call to `run`, but a consistent value during a single `run` (`out1` and `out2` receive the same random input

In [ ]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

In [ ]:
# resetting graph
# see more in documentation
sess.close()
tf.reset_default_graph()
sess = tf.Session(config=config)

In [ ]:
# reset tensorboard
!rm log/* -rf
writer = tf.summary.FileWriter('log/')

## Feeding

A graph can be parameterized to accept external inputs, known as **placeholders**. A **placeholder** is a like a function argument.

In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [ ]:
writer.add_graph(tf.get_default_graph())

In [ ]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

Note that the `feed_dict` argument can be used to overwrite any tensor in the graph. The only difference between placeholders and other `tf.Tensors` is that placeholders throw an error if no value is fed to them.

## Evaluating Tensors

Once the computation graph has been built, you can run the computation that produces a particular tf.Tensor and fetch the value assigned to it. This is often useful for debugging as well as being required for much of TensorFlow to work.

The simplest way to evaluate a Tensor is using the Tensor.eval method. For example:

In [ ]:
p = tf.placeholder(tf.float32)
t = p + 1.0
print(t.eval(feed_dict={p:1},
             session=sess))

## Some tensor examples

#### Rank

In [ ]:
# rank 0
mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

In [ ]:
# rank 1
mystr = tf.Variable(["Hello"], tf.string)
cool_numbers  = tf.Variable([3.14159, 2.71828], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([12.3 - 4.85j, 7.5 - 6.23j], tf.complex64)

In [ ]:
# higher ranks
mymat = tf.Variable([[7],[11]], tf.int16)
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
linear_squares = tf.Variable([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.Variable([ [4, 9], [16, 25] ], tf.int32)
rank_of_squares = tf.rank(squarish_squares)
mymatC = tf.Variable([[7],[11]], tf.int32)

# pictures x height x width x color
my_image = tf.zeros([10, 299, 299, 3])  

In [ ]:
r = tf.rank(my_image)
# After the graph runs, r will hold the value 4.
r.eval(session=sess)

#### slices

In [ ]:
my_vector = tf.placeholder(tf.int32, shape=None, name="my_vector")

In [ ]:
with sess.as_default():
    print(my_vector[2:4].eval(feed_dict={my_vector: np.arange(10)}))
    
    print(my_image[1, 3:10, 2, 1:].eval().shape)

#### shape

In [ ]:
zeros = tf.zeros(my_image[:,3:10, ...].shape[1])
print(zeros.eval(session=sess).shape, )
print(zeros.eval(session=sess))

#### reshape

In [ ]:
rank_three_tensor = tf.ones([3, 4, 5])

# Reshape existing content into a 6x10 matrix
matrix = tf.reshape(rank_three_tensor, [6, 10])  

# Reshape existing content into a 3x20 matrix.
#  -1 tells reshape to calculate the size of this dimension.
matrixB = tf.reshape(matrix, [3, -1])  

# Reshape existing content into a 4x3x5 tensor
matrixAlt = tf.reshape(matrixB, [4, 3, -1])  

#### reduce ...

In [ ]:
np.random.seed(1)
a = tf.constant(
    np.random.randint(0, 10, (3, 4))
)

In [ ]:
sess.run(a)

In [ ]:
sess.run(tf.reduce_sum(a))

In [ ]:
sess.run(tf.reduce_sum(a, axis=-1))

In [ ]:
sess.run(tf.reduce_mean(a, axis=0, keepdims=True))

In [ ]:
sess.run(tf.reduce_prod(a, keepdims=True))

### Variables

In [ ]:
a = tf.Variable(initial_value=1)

In [ ]:
sess.run(a)

In [ ]:
sess.run(tf.global_variables_initializer())
# sess.run(a.initializer)
a.initializer

In [ ]:
sess.run(a)

In [ ]:
op_assign = a.assign(3)
op_assign_add = a.assign_add(2) # same as a.assign(a + 2)
op_assign_sub = a.assign_sub(-2)

sess.run([op_assign.op, a])

## Some cool stuff: Gradients

`tf.gradients(ys, xs)` Constructs symbolic partial derivatives of sum of ys w.r.t. x in xs.

In [ ]:
# for tensorborad
sess.close()
tf.reset_default_graph()
sess = tf.Session(config=config)
!rm log/* -rf
writer = tf.summary.FileWriter('log/')

### Simplest example
## $ y = x^3 $

In [ ]:
x = tf.placeholder(dtype=tf.float64, shape=(), name='x')
y = x**3

In [ ]:
y.eval({x: 2}, session=sess)

In [ ]:
grad_y = tf.gradients(y, x, name="grad_y")
grad_y

## $ \frac{dy}{dx} = 3\, x^2 $

In [ ]:
grad_y[0].eval({x: 2}, session=sess)

In [ ]:
writer.add_graph(tf.get_default_graph())

Look at tensorboard (refresh the tensorboard tab)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
X = np.linspace(-3, 3, 100)
with sess.as_default():
    Y = np.array([y.eval({x: _x}) for _x in X])
    grad_Y = np.array([grad_y[0].eval({x: _x}) for _x in X])
plt.figure(figsize=(8, 5))
plt.plot(X, Y, label="function")
plt.plot(X, grad_Y, label="gradient")
plt.grid()
plt.legend()

### More complicated example

In [ ]:
my_scalar = tf.placeholder(dtype=tf.float64)
my_vector = tf.placeholder(tf.float64)

#Compute the gradient of the next weird function over my_scalar and my_vector
#warning! Trying to understand the meaning of that function may result in permanent brain damage

weird_psychotic_function = tf.reduce_mean(
    (my_vector+my_scalar)**(1+tf.nn.moments(my_vector, axes=[0])[1]) + \
    1. / tf.log(my_scalar + tf.sqrt(my_scalar**2 + 1))) / (my_scalar**2 + 1) + \
    0.01 * tf.sin(2 * my_scalar**1.5) * (tf.reduce_sum(my_vector) * my_scalar**2) * \
    tf.exp((my_scalar - 4)**2) / (1 + tf.exp((my_scalar - 4)**2)) * \
    (1. - (tf.exp( - (my_scalar - 4)**2)) / (1 + tf.exp( - (my_scalar - 4)**2)))**2

In [ ]:
der_by_scalar, der_by_vector = (None, None) #<student.compute_grad_over_scalar_and_vector()>
der_by_scalar, der_by_vector = tf.gradients(weird_psychotic_function, [my_scalar, my_vector])

compute_weird_function = lambda x, y: sess.run(weird_psychotic_function, {my_scalar : x, my_vector : y })
compute_der_by_scalar = lambda x, y: sess.run(der_by_scalar, {my_scalar : x, my_vector : y })

In [ ]:
compute_der_by_scalar_numerically = lambda x, y, dx: (
    (sess.run(weird_psychotic_function, {my_scalar : x + dx, my_vector : y }) - 
     sess.run(weird_psychotic_function, {my_scalar : x, my_vector : y })) /dx)

In [ ]:
#Plotting your derivative
vector_0 = [1, 2, 3]

scalar_space = np.linspace(0.5, 7, 100)

y = [compute_weird_function(x, vector_0) for x in scalar_space]
y_der_by_scalar = [compute_der_by_scalar(x, vector_0) for x in scalar_space]
y_der_by_scalar_numerically = [compute_der_by_scalar_numerically(x, vector_0, 0.05) for x in scalar_space]

plt.figure(figsize=(15, 10))

plt.plot(scalar_space, y, label='function')
plt.plot(scalar_space, y_der_by_scalar, label='derivative')
plt.plot(scalar_space, y_der_by_scalar_numerically, ":", lw=4, label='derivative numerically')
plt.grid()
plt.legend()